In [1]:
! pip install ko-sentence-transformers
# BERT - SBERT (senetence)

  Created wheel for ko-sentence-transformers: filename=ko_sentence_transformers-0.3-py3-none-any.whl size=9680 sha256=5a034ecbbf0b95c3e0406c2f9c180181964b46851130bf1510d60ea936324e4f
  Stored in directory: c:\users\samsung\appdata\local\pip\cache\wheels\8f\46\7f\b938df186091c9ddc53e01afbfe7a889aa21159d811ebfd377
Successfully built ko-sentence-transformers
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2022.2.0
    Uninstalling fsspec-2022.2.0:
      Successfully uninstalled fsspec-2022.2.0


In [1]:
from sentence_transformers import SentenceTransformer, util
import numpy as np
import re

embedder = SentenceTransformer("jhgan/ko-sbert-sts")
# korean-sbert를 불러옵니다.

# Corpus with example sentences
corpus = ['한 남자가 음식을 먹는다.',
 '한 남자가 빵 한 조각을 먹는다.',
 '그 여자가 아이를 돌본다.',
 '한 남자가 말을 탄다.',
 '한 여자가 바이올린을 연주한다.',
 '두 남자가 수레를 숲 속으로 밀었다.',
 '한 남자가 담으로 싸인 땅에서 백마를 타고 있다.',
 '원숭이 한 마리가 드럼을 연주한다.',
 '치타 한 마리가 먹이 뒤에서 달리고 있다.']
# 한국어로 fine tuning된 sbert 모델로, 긴 문장 혹은 단어 모두 clustering이 가능합니다.

corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

# Query sentences:
queries = ['한 남자가 파스타를 먹는다.',
  '고릴라 의상을 입은 누군가가 드럼을 연주하고 있다.',
  '치타가 들판을 가로 질러 먹이를 쫓는다.']
# 해당 세 문장이 구분할 문장입니다.

# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = 5
for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)
    cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    cos_scores = cos_scores.cpu()

    #We use np.argpartition, to only partially sort the top_k results
    top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for idx in top_results[0:top_k]:
        print(corpus[idx].strip(), "(Score: %.4f)" % (cos_scores[idx]))

# cossin similarity로 score를 측정하여 가장 큰 값들을 보여줍니다. 여기서는 가장 큰 값 5개를 보여주며 각 queries의 문장들을 기준으로 corpus의 어떤 문장들이 가장 유사한지를 보여줍니다.

c:\Users\SAMSUNG\anaconda3\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(






Query: 한 남자가 파스타를 먹는다.

Top 5 most similar sentences in corpus:
한 남자가 음식을 먹는다. (Score: 0.6154)
한 남자가 빵 한 조각을 먹는다. (Score: 0.5451)
한 여자가 바이올린을 연주한다. (Score: 0.0980)
한 남자가 말을 탄다. (Score: 0.0686)
한 남자가 담으로 싸인 땅에서 백마를 타고 있다. (Score: 0.0572)




Query: 고릴라 의상을 입은 누군가가 드럼을 연주하고 있다.

Top 5 most similar sentences in corpus:
원숭이 한 마리가 드럼을 연주한다. (Score: 0.7253)
치타 한 마리가 먹이 뒤에서 달리고 있다. (Score: 0.1940)
한 여자가 바이올린을 연주한다. (Score: 0.1483)
한 남자가 담으로 싸인 땅에서 백마를 타고 있다. (Score: 0.0471)
한 남자가 음식을 먹는다. (Score: 0.0082)




Query: 치타가 들판을 가로 질러 먹이를 쫓는다.

Top 5 most similar sentences in corpus:
치타 한 마리가 먹이 뒤에서 달리고 있다. (Score: 0.7887)
원숭이 한 마리가 드럼을 연주한다. (Score: 0.3887)
두 남자가 수레를 숲 속으로 밀었다. (Score: 0.1610)
한 남자가 음식을 먹는다. (Score: 0.0739)
그 여자가 아이를 돌본다. (Score: 0.0568)


In [ ]:
# 즉, 해당 문장과 단어 사이의 vecotr 값을 구분짓는 것이 아니라
# 각 문장과 단어의 의미를 기준으로 구분하며
# 이를 clustering으로 사용 가능할 것으로 보입니다.
# 본래는 sts로 문장의 sentiment를 기준으로 유사한 문장들을 보여주는 역할입니다.
# 하지만, 이 프로젝트에서는 해당 ko-sentence-transformer를 clustering에서도 사용할 수 있을거라 기대됩니다.